<a href="https://colab.research.google.com/github/chw8207/NLP-study/blob/main/ch02_%EB%B9%88%EB%8F%84%EB%B6%84%EC%84%9D_%EC%9B%8C%EB%93%9C%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 필요한 라이브러리 다운로드

In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 45.4 MB/s eta 0:00:00


In [2]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 2. matplotlib에 한글 폰트 사용 설정

In [8]:
!apt -qq -y install fonts-nanum

fonts-nanum is already the newest version (20180306-3).
The following package was automatically installed and is no longer required:
  libnvidia-common-525
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [16]:
import matplotlib.font_manager as fm
from pathlib import Path
from matplotlib import rc

font_path = Path('/usr/share/fonts/truetype/nanum/NanumGothic.ttf') 
fontprop = fm.FontProperties(fname=font_path)
rc('font', family=fontprop.get_name(), size=16)
# fm._rebuild()

### 3. pandas를 활용하여 엑셀파일 데이터 읽기기

In [20]:
from glob import glob
glob('/content/drive/MyDrive/Colab Notebooks/자연어처리공부/NLP-study/NLP-study/data/*')

['/content/drive/MyDrive/Colab Notebooks/자연어처리공부/NLP-study/NLP-study/data/spam.csv',
 '/content/drive/MyDrive/Colab Notebooks/자연어처리공부/NLP-study/NLP-study/data/영화_위키피디아_최근5년.xlsx']

In [22]:
import pandas as pd
ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/자연어처리공부/NLP-study/NLP-study/data/'
df = pd.read_excel(ROOT_PATH + '영화_위키피디아_최근5년.xlsx', sheet_name='2021')
df.head()

,순위,영화명,개봉일,개요,줄거리
0,1,스파이더맨: 노 웨이 홈,2021-12-15,《스파이더맨: 노 웨이 홈》(영어: Spider-Man: No Way Home)은 ...,전작 《스파이더맨: 파 프롬 홈》에서 미스테리오가 스파이더맨이 살인자라고 누명을 씌...
1,2,모가디슈,2021-07-28,"《모가디슈》는 2021년에 개봉한 대한민국의 영화이다. 류승완 감독의 작품으로, 7...",NaN
2,3,이터널스,2021-11-03,《이터널스》(영어: The Eternals)는 2021년 공개된 미국의 슈퍼히어로 ...,데비안츠라는 괴물들로부터 인류를 지키고 인류를 진화시키기 위해 신과 같은 존재인 셀...
3,4,블랙 위도우,2021-07-07,《블랙 위도우》(영어: Black Widow)는 2021년 개봉한 미국의 슈퍼히어로...,NaN
4,5,분노의 질주: 더 얼티메이트,2021-05-19,"""분노의 질주: 더 얼티메이트""(영어: F9)는 2021년 공개된 미국의 액션 영화...",도미닉(빈 디젤)은 자신과 가장 가까웠던 형제 제이콥(존 시나)이 사이퍼(샤를리즈 ...


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순위      30 non-null     int64         
 1   영화명     30 non-null     object        
 2   개봉일     30 non-null     datetime64[ns]
 3   개요      30 non-null     object        
 4   줄거리     26 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 1.3+ KB


In [24]:
# 5개년 데이터 가져오기기
df_list = []
for year in ['2021', '2020', '2019', '2018', '2017'] : 
  df_tmp = pd.read_excel(ROOT_PATH + '영화_위키피디아_최근5년.xlsx', sheet_name=year)
  df_list.append(df_tmp)
len(df_list)

5

In [29]:
df_list[4].head()

,순위,영화명,개봉일,개요,줄거리
0,1,택시운전사,2017-08-02,"《택시운전사》는 2017년에 개봉한 대한민국의 영화이다. 실화를 바탕으로 하였으며,...","1980년 5월, 서울에서 택시기사일을 하며 하나 뿐인 딸과 착실하게 살아가는 만섭..."
1,2,신과함께-죄와 벌,2017-12-20,"《신과함께-죄와 벌》은 2017년 12월 20일에 개봉한 대한민국의 영화로, 주호민...",김자홍(차태현)이 삼차사와 49일 동안 7개의 지옥에서 7개의 재판을 받으며 자신의...
2,3,공조,2017-01-18,《공조》는 2017년에 개봉한 대한민국의 영화이다.,특수부대 출신 형사 림철령은 작전수행 도중 지원을 기다리자는 동료들의 말도 무시하고...
3,4,스파이더맨: 홈 커밍,2017-07-05,《스파이더맨: 홈커밍》(영어: Spider-Man: Homecoming)은 마블 코...,-
4,5,범죄도시,2017-10-03,"""범죄도시""는 2017년에 개봉한 대한민국의 영화이다.\n\n서울 구로구 가리봉동 ...","통쾌하고, 화끈하고, 살벌하게 나쁜놈들 때려잡는 강력반 형사들의 ‘조폭소탕작전’ 이..."


In [30]:
# 5개년 데이터 하나로 합치기 
df = pd.concat(df_list)
df.shape

(150, 5)